# Autoregressive covariance structure 

We try to parameterize covariance Gamma only using two parameters rho and sigma2 as in the AR(1) structure.

For n = 50 this autoregressive model converges in 31 iterations using Quasi-Newton

1. First we need to modify the structure itself
2. Second modify the loglikelihood function 
3. Third modify gradient and hessian with respect to beta
4. ADD gradient and hessian with respect to AR(1) parameterization
5. Quasi-Newton works, we need to fix and make Newton work

To Do: Strangely Newton's takes more iterations than Quasi-Newton... Look into the Hessians?

In [1]:
using DataFrames, Random, GLM, GLMCopula, Test
using LinearAlgebra, BenchmarkTools, Revise

Random.seed!(1234)

# sample size
N = 10000
# observations per subject
n = 50
ρ = 0.1
σ2 = 0.1

V = zeros(n, n) # will store the AR(1) structure without sigma2

mean = 5

dist = Poisson

V = get_AR_cov(n, ρ, σ2, V)

# true Gamma
Γ = σ2 * V

┌ Info: Precompiling GLMCopula [c47b6ae2-b804-4668-9957-eb588c99ffbc]
└ @ Base loading.jl:1278


50×50 Array{Float64,2}:
 0.1      0.01     0.001    0.0001   …  1.0e-47  1.0e-48  1.0e-49  1.0e-50
 0.01     0.1      0.01     0.001       1.0e-46  1.0e-47  1.0e-48  1.0e-49
 0.001    0.01     0.1      0.01        1.0e-45  1.0e-46  1.0e-47  1.0e-48
 0.0001   0.001    0.01     0.1         1.0e-44  1.0e-45  1.0e-46  1.0e-47
 1.0e-5   0.0001   0.001    0.01        1.0e-43  1.0e-44  1.0e-45  1.0e-46
 1.0e-6   1.0e-5   0.0001   0.001    …  1.0e-42  1.0e-43  1.0e-44  1.0e-45
 1.0e-7   1.0e-6   1.0e-5   0.0001      1.0e-41  1.0e-42  1.0e-43  1.0e-44
 1.0e-8   1.0e-7   1.0e-6   1.0e-5      1.0e-40  1.0e-41  1.0e-42  1.0e-43
 1.0e-9   1.0e-8   1.0e-7   1.0e-6      1.0e-39  1.0e-40  1.0e-41  1.0e-42
 1.0e-10  1.0e-9   1.0e-8   1.0e-7      1.0e-38  1.0e-39  1.0e-40  1.0e-41
 1.0e-11  1.0e-10  1.0e-9   1.0e-8   …  1.0e-37  1.0e-38  1.0e-39  1.0e-40
 1.0e-12  1.0e-11  1.0e-10  1.0e-9      1.0e-36  1.0e-37  1.0e-38  1.0e-39
 1.0e-13  1.0e-12  1.0e-11  1.0e-10     1.0e-35  1.0e-36  1.0e-37  1.0e-38
 

In [2]:
vecd = [dist(mean) for i in 1:n]
nonmixed_multivariate_dist = NonMixedMultivariateDistribution(vecd, Γ)

Y_Nsample = simulate_nobs_independent_vectors(nonmixed_multivariate_dist, N)

10000-element Array{Array{Float64,1},1}:
 [7.0, 8.0, 7.0, 4.0, 8.0, 2.0, 6.0, 6.0, 6.0, 7.0  …  5.0, 3.0, 4.0, 6.0, 4.0, 5.0, 5.0, 6.0, 4.0, 7.0]
 [3.0, 7.0, 6.0, 3.0, 6.0, 6.0, 3.0, 3.0, 8.0, 5.0  …  3.0, 4.0, 5.0, 8.0, 2.0, 5.0, 4.0, 4.0, 8.0, 6.0]
 [6.0, 3.0, 3.0, 7.0, 4.0, 3.0, 6.0, 2.0, 1.0, 8.0  …  2.0, 4.0, 3.0, 4.0, 6.0, 3.0, 4.0, 4.0, 3.0, 5.0]
 [9.0, 4.0, 5.0, 3.0, 5.0, 5.0, 4.0, 12.0, 4.0, 5.0  …  4.0, 8.0, 7.0, 9.0, 5.0, 3.0, 8.0, 5.0, 4.0, 4.0]
 [5.0, 7.0, 4.0, 7.0, 4.0, 1.0, 7.0, 3.0, 5.0, 4.0  …  9.0, 7.0, 3.0, 1.0, 3.0, 5.0, 3.0, 6.0, 5.0, 5.0]
 [6.0, 6.0, 10.0, 2.0, 4.0, 2.0, 6.0, 7.0, 1.0, 5.0  …  2.0, 6.0, 3.0, 6.0, 3.0, 6.0, 4.0, 4.0, 4.0, 5.0]
 [5.0, 4.0, 6.0, 3.0, 4.0, 6.0, 0.0, 3.0, 3.0, 3.0  …  4.0, 6.0, 3.0, 3.0, 5.0, 1.0, 4.0, 1.0, 5.0, 10.0]
 [6.0, 4.0, 5.0, 7.0, 5.0, 8.0, 5.0, 3.0, 10.0, 1.0  …  5.0, 4.0, 5.0, 3.0, 4.0, 6.0, 5.0, 4.0, 7.0, 2.0]
 [4.0, 5.0, 8.0, 5.0, 5.0, 4.0, 7.0, 3.0, 3.0, 6.0  …  5.0, 2.0, 4.0, 7.0, 6.0, 5.0, 6.0, 5.0, 7.0, 4.0]
 [4.0, 6.0

In [3]:
Random.seed!(1234)

d = Poisson()
link = LogLink()
D = typeof(d)
Link = typeof(link)
T = Float64
gcs = Vector{GLMCopulaARObs{T, D, Link}}(undef, N)

for i in 1:N
    y = Float64.(Y_Nsample[i])
    X = ones(n, 1)
    gcs[i] = GLMCopulaARObs(y, X, d, link)
end

gcm = GLMCopulaARModel(gcs);

In [4]:
initialize_model!(gcm)
@show gcm.β
@show exp.(gcm.β);

initializing β using Newton's Algorithm under Independence Assumption
1 0.0 -1.1093585944799678e6 39999
2 -1.1093585944799678e6 -1.1093585944799678e6 39999
gcm.β = [1.613054963012508]
exp.(gcm.β) = [5.018117999999999]


In [5]:
fill!(gcm.ρ, 0.0)
fill!(gcm.σ2, 0.0)
loglikelihood!(gcm, true, true)

gcm2 = deepcopy(gcm);

In [6]:
@time GLMCopula.fit!(gcm, IpoptSolver(print_level = 5, max_iter = 100, warm_start_init_point="yes", hessian_approximation = "limited-memory"))

gcm.θ = [1.613054963012508, 0.0, 0.0]
gcm.∇θ = [2.5979574047596543e-10, 0.0, 7034.695979448923]
gcm.θ = [1.613054963012508, 0.00999999, 0.00999999]
gcm.∇θ = [-2013.5757220422408, 10.383726613141, 3052.1035044366386]

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:       

-1.1092594676023405e6

In [7]:
@time GLMCopula.fit!(gcm2, IpoptSolver(print_level = 5, max_iter = 100, warm_start_init_point="yes", hessian_approximation = "exact"))

gcm.θ = [1.613054963012508, 0.0, 0.0]
gcm.∇θ = [2.5979574047596543e-10, 0.0, 7034.695979448923]
gcm.θ = [1.613054963012508, 0.00999999, 0.00999999]
gcm.∇θ = [-2013.5757220422408, 10.383726613141, 3052.1035044366386]
This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        3

Total number of variables............................:        3
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        2
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0


  31  1.1092595e+06 0.00e+00 1.29e-04  -5.7 4.60e-07    -  1.00e+00 1.00e+00f  1
gcm.θ = [1.6102620414591533, 0.09493562984186843, 0.09341037332741652]
gcm.∇θ = [0.0033610023755414886, -0.001205873603157992, -0.0011518792032525482]
  32  1.1092595e+06 0.00e+00 4.78e-05  -5.7 1.57e-07    -  1.00e+00 1.00e+00f  1
gcm.θ = [1.6102620428468521, 0.09493567546601907, 0.09341041522305643]
gcm.∇θ = [-0.0008959530070473676, -0.001228421906533106, -0.001281318053266034]
  33  1.1092595e+06 0.00e+00 1.27e-05  -5.7 4.56e-08    -  1.00e+00 1.00e+00f  1
gcm.θ = [1.610262042476929, 0.09493253171978579, 0.09340984708035971]
gcm.∇θ = [0.012153630563862983, 6.308388785880581e-5, 0.0003648652110479489]
  34  1.1092595e+06 0.00e+00 1.73e-04  -8.6 3.14e-06    -  1.00e+00 1.00e+00f  1
gcm.θ = [1.6102620474949483, 0.09493269764783889, 0.0934100098256865]
gcm.∇θ = [-0.0034819682131876917, -2.010377737673985e-5, -0.0001286436341612074]
  35  1.1092595e+06 0.00e+00 4.95e-05  -8.6 1.66e-07    -  1.00e+00 1.00e+00

-1.109259467602342e6

In [8]:
@show gcm.∇θ
@show gcm2.∇θ;

gcm.∇θ = [4.5257527414577225e-7, 3.9946426583226113e-8, -1.5506833150880084e-8]
gcm2.∇θ = [-3.019433965079088e-7, -1.6637508584438354e-6, -1.7247190573499438e-6]


In [9]:
@show gcm.θ
@show gcm2.θ;

gcm.θ = [1.610262046457385, 0.09493265918554418, 0.09340996458144848]
gcm2.θ = [1.6102620464524542, 0.09493266337784918, 0.09340996517514197]


In [10]:
@show loglikelihood!(gcm, true, true)
@show loglikelihood!(gcm2, true, true);

loglikelihood!(gcm, true, true) = -1.1092594676023405e6
loglikelihood!(gcm2, true, true) = -1.109259467602342e6
